# `SHORT VERSION`: Send Data Python -> PSQL

# <font color=red> Mr Fugu Data Science </font>

# (◕‿◕✿)

# Purpose & Outcome:

+ Create a connection to PSQL 
+ send CSV and data frames to PSQL from Python using (`psycopg2`)
+ Send data back to python from postgresql
+ show some simple queries

`------------------------------------`

Install Psycopg2:
**`python -m pip install psycopg2`**

if this doesn't work, try changing pip to pip3 depending on what version of Python you are using

if that doesn't work either try to do: conda install -c anaconda psycopg2

For my installation I found problems with installation: I did two steps: pip3 install psycopg2 followed by the conda install -c anaconda psycopg2. The first install showed complete but did not work, I feel that I may have been imcomplete with all the dependencies needed. Also, I was getting a Python 2.7 error hashing. I suggest that you first check the version of Python and Location PATH. I feel that my default Location may be anaconda calling Python if I remember correctly.

I thought this was from using a wrong default version of Python, changed my default and still didn't work. That was not the entire case, try the above and hope this works for you.

Separate source for installing: depending on situation. [`Install Mac`](https://www.youtube.com/watch?v=N4RxnQH2pVY) |
[`Windows Install`](https://kb.objectrocket.com/postgresql/how-to-install-psycopg2-in-windows-1460)

In [1]:
import psycopg2             # python -> psql connection
import psycopg2.extras

import pandas as pd         # create dataframes 

# Import the 'config' function from the config_user_dta.py file:
from config_user_dta import config

`If you do not want to create init or config files`: do something similar to this

**import psycopg2**

**conn = psycopg2.connect("dbname=test user=postgres")** # Connect to an existing database

**cur = conn.cursor( )** #Open a cursor to perform database operations

In [55]:
# Establish a connection to the database by creating a cursor object

# Get the config params
params_ = config()

# Connect to the Postgres_DB:
conn = psycopg2.connect(**params_)

# Create new_cursor allowing us to write Python to execute PSQL:
cur = conn.cursor()

conn.autocommit = True  # read documentation understanding when to Use & NOT use (TRUE)

# Cursor and Connection are Context Managers :
+ allowing you to use the with statement, and psycopg2 will commit transaction unless there is an error

[further reading](https://www.postgresqltutorial.com/postgresql-python/transaction/)

In [47]:
# Our Data:

addr_df=pd.read_csv('fake_users_R.csv')
addr_df.reset_index(drop=True,inplace=True)
addr_df_=addr_df.drop('Unnamed: 0',axis=1)
addr_df_.head()

,credit_card,email,first_name,last_name,primary_phone_number
0,5399-3484-4724-7187,gso@qiegan.sqe,Donyell Ann,Ospina,5219459148
1,1630-5261-6108-7631,xnji@gfruaxqnvm.fha,Bishop,Siyed,4164254716
2,4435-3866-1076-3595,dvyco@tkzhsop.zxg,Connor,Powers,3627413915
3,3489-7099-9906-8660,fy@uvfhplatmz.cam,Kylie,Her,3562764561
4,8631-4500-5666-1510,rztkvliou@dkeinhgysf.deo,Anthony,Vo,7345795348


In [79]:

def create_staging_table(cursor):
    cursor.execute("""
        DROP TABLE IF EXISTS staging_fake_ppl CASCADE;
        CREATE UNLOGGED TABLE staging_fake_ppl (
            credit_card         TEXT,
            email               TEXT,
            first_name          TEXT,
            last_name           TEXT,
            primary_phone       TEXT
        );""")

# look at the documentation of PSQL (UNLOGGED TABLE vs TEMP)
# CASCADE was used because I had done this before and there were dependencies I needed
# to remove

In [80]:
# creating our schema  and sending the table to psql
with conn.cursor() as cursor:
    create_staging_table(cursor)

# `Send .CSV( )`: Python --> PSQL

In [81]:

# addr_df_.to_csv('address_Python_convertR.csv',index=False)

def send_csv_to_psql(connection,csv,table_):
    sql = "COPY %s FROM STDIN WITH CSV HEADER DELIMITER AS ','"
    file = open(csv, "r")
    table = table_
    with connection.cursor() as cur:
        cur.execute("truncate " + table + ";")  #avoiding uploading duplicate data!
        cur.copy_expert(sql=sql % table, file=file)
        connection.commit()
#         cur.close()
#         connection.close()
    return connection.commit()

send_csv_to_psql(conn,'address_Python_convertR.csv','staging_fake_ppl')

# Simple Query with Psycog2:

+ when doing a SELECT query use: fetchone( ), fetchall( ) or fetchmany( ) methods




In [82]:
sql_="SELECT COUNT(*) FROM staging_fake_ppl"
cur.execute(sql_)
cur.fetchone()

(5826,)

In [83]:
# do another query:

cur.execute("SELECT * FROM staging_fake_ppl LIMIT 4")
cur.fetchall()

[('5399-3484-4724-7187',
  'gso@qiegan.sqe',
  'Donyell Ann',
  'Ospina',
  '5219459148'),
 ('1630-5261-6108-7631',
  'xnji@gfruaxqnvm.fha',
  'Bishop',
  'Siyed',
  '4164254716'),
 ('4435-3866-1076-3595',
  'dvyco@tkzhsop.zxg',
  'Connor',
  'Powers',
  '3627413915'),
 ('3489-7099-9906-8660', 'fy@uvfhplatmz.cam', 'Kylie', 'Her', '3562764561')]

# Alternate Way to Query:

+ Quick and dirty way to Query PSQL and bring data into Python as a DF

In [72]:

import pandas.io.sql as sqlio
# conn = psycopg2.connect("host='{}' port={} dbname='{}' user={} password={}".format(host, port, dbname, username, pwd))
# sql = "select count(*) from staging_fake_ppl;"
sql = "select * from staging_fake_ppl;"

dat = sqlio.read_sql_query(sql, conn)

dat.head()

,credit_card,email,first_name,last_name,primary_phone
0,5399-3484-4724-7187,gso@qiegan.sqe,Donyell Ann,Ospina,5219459148
1,1630-5261-6108-7631,xnji@gfruaxqnvm.fha,Bishop,Siyed,4164254716
2,4435-3866-1076-3595,dvyco@tkzhsop.zxg,Connor,Powers,3627413915
3,3489-7099-9906-8660,fy@uvfhplatmz.cam,Kylie,Her,3562764561
4,8631-4500-5666-1510,rztkvliou@dkeinhgysf.deo,Anthony,Vo,7345795348


# `Convert Df --> List(Dict())` : then sending from 

# Python--> PSQL

In [61]:
def create_staging_table_(cursor):
    cursor.execute("""
        DROP TABLE IF EXISTS staging_fake_ppl02;
        CREATE UNLOGGED TABLE staging_fake_ppl02 (
            credit_card          TEXT,
            email                TEXT,
            first_name           TEXT,
            last_name            TEXT,
            primary_phone_number TEXT
        );""")

In [62]:
with conn.cursor() as cursor:
    create_staging_table_(cursor)

In [63]:
def fcn(df,table,cur):

    if len(df) > 0:
        df_columns = list(df)
        # create (col1,col2,...)
        columns = ",".join(df_columns)

        # create VALUES('%s', '%s",...) one '%s' per column
        values = "VALUES({})".format(",".join(["%s" for _ in df_columns])) 

        #create INSERT INTO table (columns) VALUES('%s',...)
        insert_stmt = "INSERT INTO {} ({}) {}".format(table,columns,values)
        cur.execute("truncate " + table + ";")  # avoiding uploading duplicate data!
        cur = conn.cursor()
        psycopg2.extras.execute_batch(cur, insert_stmt, df.values)
    conn.commit()

In [64]:
fcn(addr_df_,'staging_fake_ppl02',cur)
# cur.close()
# conn.close()

In [71]:
# Quick Query: to check data, it will be the same as other file for similar table

cur.execute("select * from staging_fake_ppl02 limit 6")
cur.fetchall()

[('5399-3484-4724-7187',
  'gso@qiegan.sqe',
  'Donyell Ann',
  'Ospina',
  '5219459148'),
 ('1630-5261-6108-7631',
  'xnji@gfruaxqnvm.fha',
  'Bishop',
  'Siyed',
  '4164254716'),
 ('4435-3866-1076-3595',
  'dvyco@tkzhsop.zxg',
  'Connor',
  'Powers',
  '3627413915'),
 ('3489-7099-9906-8660', 'fy@uvfhplatmz.cam', 'Kylie', 'Her', '3562764561'),
 ('8631-4500-5666-1510',
  'rztkvliou@dkeinhgysf.deo',
  'Anthony',
  'Vo',
  '7345795348'),
 ('1459-9918-1722-7369',
  'jofmezlbp@iw.evx',
  'Mutammam',
  'Mares',
  '3247247289')]

# <font color=red>LIKE</font>, Share & 

# <font color=red>SUB</font>scribe

# Citations & Help:

# ◔̯◔

https://www.datacamp.com/community/tutorials/tutorial-postgresql-python

https://hackersandslackers.com/psycopg2-postgres-python/

https://pynative.com/python-postgresql-tutorial/

https://stackoverflow.com/questions/23103962/how-to-write-dataframe-to-postgres-table

https://alvinalexander.com/blog/post/postgresql/log-in-postgresql-database/ (command line short cuts)

https://stackoverflow.com/questions/35651586/psycopg2-cursor-already-closed 